In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
from torch.utils.data import Dataset, DataLoader
import glob
import wandb
import os
import torch.optim as optimizers

In [3]:
import dfs_code
from torch_geometric.data import InMemoryDataset, Data
import pickle
import torch
import torch.nn as nn
import tqdm
import copy
import pandas as pd
#torch.multiprocessing.set_sharing_strategy('file_system') # this is important on local machine
#def set_worker_sharing_strategy(worker_id: int) -> None:
#    torch.multiprocessing.set_sharing_strategy('file_system')

In [4]:
import sys
sys.path = ['../../../src'] + sys.path
from dfs_transformer import EarlyStopping, DFSCodeSeq2SeqFC, smiles2graph, PubChem

In [5]:
wandb.init(project='pubchem-bert', entity='chrisxx', name='bert10M')

config = wandb.config
config.mode = "min2min" #rnd2rnd
config.fraction_missing = 0.15
config.n_atoms = 118
config.n_bonds = 5
config.emb_dim = 120
config.nhead = 12
config.nlayers = 6
config.max_nodes = 250
config.max_edges = 500
config.dim_feedforward = 2048
config.n_files = 64
config.n_splits = 16
config.n_iter_per_split = 1
config.lr = 0.00003
config.n_epochs = 10000
config.lr_adjustment_period = 500
config.patience = 5
config.factor = 0.8
config.minimal_lr = 6e-8
config.batch_size = 50
config.accumulate_grads = 2
config.valid_patience = 100
config.valid_minimal_improvement=0.00
config.model_dir = "../../../models/new/pubchem10M/features_selfattention/medium/"
#config.data_dir = "/mnt/project/pubchem_noH/"
config.data_dir = "/home/wendlerc/noH/timeout60_4/"
#config.data_dir = "/home/wendlerc/noH/10K/"
config.pretrained_dir = "../../../models/pubchem10M/features_selfattention/medium/"#"../../models/chembl/better_transformer/medium/"
config.num_workers = 0
config.prefetch_factor = 2
config.persistent_workers = False
config.load_last = False
config.gpu_id = 1
config.missing_value = -1

wandb: Currently logged in as: chrisxx (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.12.1 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
2021-09-12 01:28:15.323990: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-09-12 01:28:15.324018: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [6]:
os.makedirs(config.model_dir, exist_ok=True)

In [7]:
path = config.data_dir

In [8]:
def collate_fn_features(dlist):
    node_batch = [] 
    edge_batch = []
    rnd_code_batch = []
    min_code_batch = []
    for d in dlist:
        rnd_code, rnd_index = dfs_code.rnd_dfs_code_from_torch_geometric(d, 
                                                                         d.z.numpy().tolist(), 
                                                                         np.argmax(d.edge_attr.numpy(), axis=1))
        node_batch += [d.node_features]
        edge_batch += [d.edge_features]
        rnd_code_batch += [torch.tensor(rnd_code)]
        min_code_batch += [d.min_dfs_code]
    return rnd_code_batch, node_batch, edge_batch, min_code_batch

In [9]:
ngpu=1
device = torch.device('cuda:%d'%config.gpu_id if (torch.cuda.is_available() and ngpu > 0) else 'cpu')

In [10]:
to_cuda = lambda T: [t.to(device) for t in T]

In [11]:
dataset = PubChem(path, n_used = 1)
loader = DataLoader(dataset, batch_size=config.batch_size, shuffle=True, pin_memory=False, collate_fn=collate_fn_features,
                   num_workers=config.num_workers, prefetch_factor=config.prefetch_factor, 
                    persistent_workers=config.persistent_workers)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 156181/156181 [00:14<00:00, 10926.23it/s]


In [12]:
data = next(iter(loader))

In [13]:
n_node_features = data[1][0].shape[1]
n_edge_features = data[2][0].shape[1]

In [14]:
print(n_node_features, n_edge_features)

133 14


In [15]:
codes = data[-1]

In [16]:
codes

[tensor([[ 0,  1,  6,  0,  6, 18, 38, 17],
         [ 1,  2,  6,  0,  6, 17, 36, 16],
         [ 2,  3,  6,  0,  6, 16, 34, 15],
         [ 3,  4,  6,  0,  6, 15, 33, 20],
         [ 4,  5,  6,  0, 16, 20, 43, 19],
         [ 5,  0, 16,  0,  6, 19, 40, 18],
         [ 3,  6,  6,  0,  7, 15, 31, 14],
         [ 6,  7,  7,  0,  6, 14, 29,  2],
         [ 7,  8,  6,  0,  7,  2,  4,  3],
         [ 8,  9,  7,  0,  6,  3,  7,  4],
         [ 9, 10,  6,  0,  6,  4,  9,  5],
         [10, 11,  6,  0,  6,  5, 11,  6],
         [10, 12,  6,  0,  8,  5, 12,  7],
         [12, 13,  8,  0,  6,  7, 15,  8],
         [13, 14,  6,  2,  6,  8, 17,  9],
         [14, 15,  6,  2,  6,  9, 20, 10],
         [15, 16,  6,  2,  6, 10, 22, 11],
         [16, 17,  6,  2,  6, 11, 24, 12],
         [17, 18,  6,  2,  6, 12, 26, 13],
         [18, 13,  6,  2,  6, 13, 27,  8],
         [ 7, 19,  6,  1,  7,  2,  3,  1],
         [19, 20,  7,  0,  6,  1,  1,  0]]),
 tensor([[ 0,  1,  6,  0,  6,  5,  9,  3],
         

In [17]:
def BERTize(codes):
    inputs = []
    targets = []
    for code in codes:
        n = len(code)
        perm = np.random.permutation(n)
        target_idx = perm[:int(config.fraction_missing*n)]
        input_idx = perm[int(config.fraction_missing*n):]
        inp = code.clone()
        target = code.clone()
        target[input_idx] = -1
        inp[target_idx] = -1
        inputs += [inp]
        targets += [target]
    return inputs, targets

In [18]:
inputs, targets = BERTize(codes)

In [19]:
inputs

[tensor([[ 0,  1,  6,  0,  6, 18, 38, 17],
         [ 1,  2,  6,  0,  6, 17, 36, 16],
         [ 2,  3,  6,  0,  6, 16, 34, 15],
         [ 3,  4,  6,  0,  6, 15, 33, 20],
         [-1, -1, -1, -1, -1, -1, -1, -1],
         [ 5,  0, 16,  0,  6, 19, 40, 18],
         [ 3,  6,  6,  0,  7, 15, 31, 14],
         [-1, -1, -1, -1, -1, -1, -1, -1],
         [ 7,  8,  6,  0,  7,  2,  4,  3],
         [-1, -1, -1, -1, -1, -1, -1, -1],
         [ 9, 10,  6,  0,  6,  4,  9,  5],
         [10, 11,  6,  0,  6,  5, 11,  6],
         [10, 12,  6,  0,  8,  5, 12,  7],
         [12, 13,  8,  0,  6,  7, 15,  8],
         [13, 14,  6,  2,  6,  8, 17,  9],
         [14, 15,  6,  2,  6,  9, 20, 10],
         [15, 16,  6,  2,  6, 10, 22, 11],
         [16, 17,  6,  2,  6, 11, 24, 12],
         [17, 18,  6,  2,  6, 12, 26, 13],
         [18, 13,  6,  2,  6, 13, 27,  8],
         [ 7, 19,  6,  1,  7,  2,  3,  1],
         [19, 20,  7,  0,  6,  1,  1,  0]]),
 tensor([[ 0,  1,  6,  0,  6,  5,  9,  3],
         

In [20]:
targets

[tensor([[-1, -1, -1, -1, -1, -1, -1, -1],
         [-1, -1, -1, -1, -1, -1, -1, -1],
         [-1, -1, -1, -1, -1, -1, -1, -1],
         [-1, -1, -1, -1, -1, -1, -1, -1],
         [ 4,  5,  6,  0, 16, 20, 43, 19],
         [-1, -1, -1, -1, -1, -1, -1, -1],
         [-1, -1, -1, -1, -1, -1, -1, -1],
         [ 6,  7,  7,  0,  6, 14, 29,  2],
         [-1, -1, -1, -1, -1, -1, -1, -1],
         [ 8,  9,  7,  0,  6,  3,  7,  4],
         [-1, -1, -1, -1, -1, -1, -1, -1],
         [-1, -1, -1, -1, -1, -1, -1, -1],
         [-1, -1, -1, -1, -1, -1, -1, -1],
         [-1, -1, -1, -1, -1, -1, -1, -1],
         [-1, -1, -1, -1, -1, -1, -1, -1],
         [-1, -1, -1, -1, -1, -1, -1, -1],
         [-1, -1, -1, -1, -1, -1, -1, -1],
         [-1, -1, -1, -1, -1, -1, -1, -1],
         [-1, -1, -1, -1, -1, -1, -1, -1],
         [-1, -1, -1, -1, -1, -1, -1, -1],
         [-1, -1, -1, -1, -1, -1, -1, -1],
         [-1, -1, -1, -1, -1, -1, -1, -1]]),
 tensor([[-1, -1, -1, -1, -1, -1, -1, -1],
         

In [21]:
model = DFSCodeSeq2SeqFC(n_atoms=config.n_atoms,
                         n_bonds=config.n_bonds, 
                         emb_dim=config.emb_dim, 
                         nhead=config.nhead, 
                         nlayers=config.nlayers, 
                         max_nodes=config.max_nodes, 
                         max_edges=config.max_edges,
                         atom_embedding=nn.Linear(n_node_features, config.emb_dim), 
                         bond_embedding=nn.Linear(n_edge_features, config.emb_dim),
                         missing_value = config.missing_value)

In [22]:
if config.pretrained_dir is not None:
    model.load_state_dict(torch.load(config.pretrained_dir+'checkpoint.pt'), strict=False)

In [23]:
if config.load_last:
    model.load_state_dict(torch.load(config.model_dir+'checkpoint.pt'))

In [24]:
optim = optimizers.Adam(model.parameters(), lr=config.lr)

lr_scheduler = optimizers.lr_scheduler.ReduceLROnPlateau(optim, mode='min', verbose=True, patience=config.patience, factor=config.factor)
#lr_scheduler = optimizers.lr_scheduler.ExponentialLR(optim, gamma=config.factor)

early_stopping = EarlyStopping(patience=config.valid_patience, delta=config.valid_minimal_improvement,
                              path=config.model_dir+'checkpoint.pt')
bce = torch.nn.BCEWithLogitsLoss()
ce = torch.nn.CrossEntropyLoss(ignore_index=-1)
softmax = nn.Softmax(dim=2)

In [25]:
model.to(device)

DFSCodeSeq2SeqFC(
  (encoder): DFSCodeEncoder(
    (emb_dfs): PositionalEncoding(
      (dropout): Dropout(p=0, inplace=False)
    )
    (emb_seq): PositionalEncoding(
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (emb_atom): Linear(in_features=133, out_features=120, bias=True)
    (emb_bond): Linear(in_features=14, out_features=120, bias=True)
    (mixer): Linear(in_features=600, out_features=600, bias=True)
    (enc): TransformerEncoder(
      (layers): ModuleList(
        (0): TransformerEncoderLayer(
          (self_attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=600, out_features=600, bias=True)
          )
          (linear1): Linear(in_features=600, out_features=2048, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
          (linear2): Linear(in_features=2048, out_features=600, bias=True)
          (norm1): LayerNorm((600,), eps=1e-05, elementwise_affine=True)
          (norm2): LayerNorm((600,), eps=1e-05, 

In [26]:
try:
    for epoch in range(config.n_epochs):
        epoch_loss = 0
        for split in range(config.n_splits):
            n_ids = config.n_files//config.n_splits
            dataset = PubChem(path, n_used = n_ids, max_nodes=config.max_nodes, max_edges=config.max_edges)
            loader = DataLoader(dataset, batch_size=config.batch_size, shuffle=True, pin_memory=False, 
                                collate_fn=collate_fn_features,
                   num_workers=config.num_workers, prefetch_factor=config.prefetch_factor, 
                    persistent_workers=config.persistent_workers)
            for j in range(config.n_iter_per_split):
                pbar = tqdm.tqdm(loader)
                for i, data in enumerate(pbar):
                    if i % config.accumulate_grads == 0: #bei 0 wollen wir das
                        optim.zero_grad()
                    rndc, nattr, eattr, minc = data
                    if config.mode == "min2min":
                        code = to_cuda(minc)
                    elif config.mode == "rnd2rnd":
                        code = to_cuda(rndc)
                    else:
                        raise ValueError("unrecognized config.mode")
                    nattr = to_cuda(nattr)
                    eattr = to_cuda(eattr)
                    #prepare inputs
                    inputs, targets = BERTize(code)
                    #prepare labels
                    targetc_seq = nn.utils.rnn.pad_sequence(targets, padding_value=-1)
                    #prediction
                    dfs1, dfs2, atm1, atm2, bnd = model(inputs, nattr, eattr)
                    pred_dfs1 = torch.reshape(dfs1, (-1, config.max_nodes))
                    pred_dfs2 = torch.reshape(dfs2, (-1, config.max_nodes))
                    pred_atm1 = torch.reshape(atm1, (-1, config.n_atoms))
                    pred_atm2 = torch.reshape(atm2, (-1, config.n_atoms))
                    pred_bnd = torch.reshape(bnd, (-1, config.n_bonds))
                    tgt_dfs1 = targetc_seq[:, :, 0].view(-1)
                    tgt_dfs2 = targetc_seq[:, :, 1].view(-1)
                    tgt_atm1 = targetc_seq[:, :, 2].view(-1)
                    tgt_atm2 = targetc_seq[:, :, 4].view(-1)
                    tgt_bnd = targetc_seq[:, :, 3].view(-1)
                    loss = ce(pred_dfs1, tgt_dfs1) 
                    loss += ce(pred_dfs2, tgt_dfs2)
                    loss += ce(pred_atm1, tgt_atm1)
                    loss += ce(pred_bnd, tgt_bnd)
                    loss += ce(pred_atm2, tgt_atm2)
                    loss.backward()
                    if (i+1) % config.accumulate_grads == 0:
                        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
                        optim.step() # bei 2 wollen wir das
                    epoch_loss = (epoch_loss*i + loss.item())/(i+1)
                    mask = tgt_dfs1 != -1
                    n_tgts = torch.sum(mask)
                    acc_dfs1 = (torch.argmax(pred_dfs1[mask], axis=1) == tgt_dfs1[mask]).sum()/n_tgts
                    acc_dfs2 = (torch.argmax(pred_dfs2[mask], axis=1) == tgt_dfs2[mask]).sum()/n_tgts
                    acc_atm1 = (torch.argmax(pred_atm1[mask], axis=1) == tgt_atm1[mask]).sum()/n_tgts
                    acc_atm2 = (torch.argmax(pred_atm2[mask], axis=1) == tgt_atm2[mask]).sum()/n_tgts
                    acc_bnd = (torch.argmax(pred_bnd[mask], axis=1) == tgt_bnd[mask]).sum()/n_tgts
                    curr_lr = list(optim.param_groups)[0]['lr']
                    wandb.log({'loss':epoch_loss, 'learning rate':curr_lr,
                               'acc-dfs1':acc_dfs1, 'acc-dfs2':acc_dfs2, 
                               'acc-atm1':acc_atm1, 'acc-atm2':acc_atm2,
                               'acc-bnd':acc_bnd})
                    pbar.set_description('Epoch %d: CE %2.6f accs: %2.2f %2.2f %2.2f %2.2f %2.2f'%(epoch+1, 
                                                                                                   epoch_loss, 
                                                                                                   100*acc_dfs1,
                                                                                                   100*acc_dfs2,
                                                                                                   100*acc_atm1,
                                                                                                   100*acc_bnd,
                                                                                                   100*acc_atm2))

                    if i % config.lr_adjustment_period == 0:
                        early_stopping(epoch_loss, model)
                        lr_scheduler.step(epoch_loss)
                        if early_stopping.early_stop:
                            break

                        if curr_lr < config.minimal_lr:
                            break
            if early_stopping.early_stop:
                break
            if curr_lr < config.minimal_lr:
                break
                
            del dataset
            del loader
            
        if early_stopping.early_stop:
            break
        if curr_lr < config.minimal_lr:
            break

except KeyboardInterrupt:
    torch.save(model.state_dict(), config.model_dir+'_keyboardinterrupt.pt')
    print('keyboard interrupt caught')


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 624736/624736 [01:05<00:00, 9522.89it/s]
Epoch 1: CE 0.334026 accs: 92.63 93.68 98.42 97.37 94.21:   4%|████▍                                                                                                           | 501/12495 [02:04<50:21,  3.97it/s]

EarlyStopping counter: 1 out of 100


Epoch 1: CE 0.330985 accs: 96.17 95.63 98.36 97.81 93.44:   8%|████████▉                                                                                                      | 1001/12495 [04:06<49:06,  3.90it/s]

EarlyStopping counter: 2 out of 100


Epoch 1: CE 0.331982 accs: 99.42 100.00 99.42 100.00 95.38:  12%|█████████████                                                                                                | 1501/12495 [06:14<45:18,  4.04it/s]

...

EarlyStopping counter: 5 out of 100


Epoch 1: CE 0.337294 accs: 97.96 96.94 98.47 98.98 95.41:  24%|██████████████████████████▋                                                                                    | 3001/12495 [12:49<36:03,  4.39it/s]

EarlyStopping counter: 6 out of 100
Epoch     7: reducing learning rate of group 0 to 2.4000e-05.


Epoch 1: CE 0.337366 accs: 100.00 98.35 98.35 98.90 93.41:  25%|███████████████████████████▉                                                                                  | 3167/12495 [13:33<39:56,  3.89it/s]


keyboard interrupt caught
